# Air-helium shock bubble interaction
This case simulates a two-dimensional helium bubble in air that interacts with a Mach 1.22 shock using the diffuse-interface method.

# Imports

In [2]:
import numpy as np
from jaxfluids import InputManager, InitializationManager, SimulationManager
from jaxfluids_postprocess import load_data, create_2D_animation, create_2D_figure
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Run simulation

In [ ]:
input_manager = InputManager("inputfiles/case_setup_diffuse.json",
                             "../numerical_setup_files/numerical_setup_diffuse.json")
initialization_manager = InitializationManager(input_manager)
sim_manager = SimulationManager(input_manager)
simulation_buffers, time_control_variables, forcing_parameters = initialization_manager.initialization()
sim_manager.simulate(simulation_buffers, time_control_variables)

# Load and Visualization

In [ ]:
path = sim_manager.output_writer.save_path_case
quantities = ["alpharho_0", "alpharho_1", "schlieren", "pressure"]
cell_centers, cell_sizes, times, data_dict = load_data(path, quantities, step=1)

density = data_dict["alpharho_0"] + data_dict["alpharho_1"]
schlieren = data_dict["schlieren"]
schlieren /= np.max(schlieren)
schlieren = np.clip(schlieren, 1e-3, 1e0)

plot_dict = {
    "schlieren": schlieren,
    "pressure": data_dict["pressure"],
    "density": density,
}

nrows_ncols = (3,1)
minmax_list = [
    [1e-3, 1e0],
    [1e5, 2.5e5],
    [0.8, 2.5]
    ]

create_2D_figure(plot_dict, times, nrows_ncols, cell_centers=cell_centers, plane="xy",
                 plane_value=0.0, minmax_list=minmax_list, dpi=300, fig_args={"figsize": (20,10)})

save_path = os.path.join(path,"images")
os.makedirs(save_path, exist_ok=True)
create_2D_animation(plot_dict, cell_centers, times, nrows_ncols=nrows_ncols,
                    save_png=save_path, fig_args={"figsize": (20,10)},
                    minmax_list=minmax_list, dpi=300)
